# OPUS Report Decoder.
> Decode NOAA NGS OPUS Reports into a Python class variable.

In [34]:
#|hide
## Code.

#|hide
### Test Data.

In [29]:
opus_txt = '''
FILE: SFKW1210.24O OP1716177177447

 1008   NOTE: You provided a zero or negative antenna height.
 1008   If ARP HGT = 0.0, OPUS solves for the position of your selected antenna's reference point (ARP).
 1008   If ARP HGT < 0.0, OPUS solves for a location inside or above the antenna
 1008
                              NGS OPUS SOLUTION REPORT
                              ========================

All computed coordinate accuracies are listed as peak-to-peak values.
For additional information: https://www.ngs.noaa.gov/OPUS/about.jsp#accuracy

      USER: lidar532@gmail.com                       DATE: May 20, 2024
RINEX FILE: sfkw121p.24o                             TIME: 03:53:37 UTC


  SOFTWARE: page5  2008.25 master253.pl 160321      START: 2024/04/30  15:51:00
 EPHEMERIS: igs23122.eph [precise]                   STOP: 2024/04/30  22:51:00
  NAV FILE: brdc1210.24n                         OBS USED: 14342 / 14604   :  98%
  ANT NAME: TRMR10-2        NONE              # FIXED AMB:    65 /    68   :  96%
ARP HEIGHT: 0.000                             OVERALL RMS: 0.014(m)


 REF FRAME: NAD_83(2011)(EPOCH:2010.0000)              ITRF2014 (EPOCH:2024.3301)

         X:     -2475273.328(m)   0.019(m)          -2475274.384(m)   0.019(m)
         Y:     -4509243.734(m)   0.013(m)          -4509242.286(m)   0.013(m)
         Z:      3760194.885(m)   0.015(m)           3760194.828(m)   0.015(m)

       LAT:   36 21  0.18896      0.022(m)        36 21  0.20210      0.022(m)
     E LON:  241 14 10.16711      0.012(m)       241 14 10.10206      0.012(m)
     W LON:  118 45 49.83289      0.012(m)       118 45 49.89794      0.012(m)
    EL HGT:         1094.647(m)   0.012(m)              1094.000(m)   0.012(m)
 ORTHO HGT:         1122.958(m)   0.068(m) [NAVD88 (Computed using GEOID18)]

                        UTM COORDINATES    STATE PLANE COORDINATES
                         UTM (Zone 11)         SPC (0404 CA 4)
Northing (Y) [meters]     4024219.772           612842.492
Easting (X)  [meters]      341727.091          2021197.302
Convergence  [degrees]    -1.04567778           0.14088889
Point Scale                0.99990865           0.99995228
Combined Factor            0.99973690           0.99978052

US NATIONAL GRID DESIGNATOR: 11SLA4172724220(NAD 83)


                              BASE STATIONS USED
PID       DESIGNATION                        LATITUDE    LONGITUDE DISTANCE(m)
DM6188 P572 SHADEQUARTCS2006 CORS GRP      N363507.843 W1185716.494   31229.8
DN7424 P567 RIOBRAVO_CS2005 CORS GRP       N352515.393 W1184512.826  103109.5
DK6413 P467 ALABAMAHILCS2006 CORS GRP      N363412.712 W1180526.193   65116.5

                 NEAREST NGS PUBLISHED CONTROL POINT
GT1835      DENNISON                       N361906.411 W1184447.601    3835.1

This position and the above vector components were computed without any
knowledge by the National Geodetic Survey regarding the equipment or
field operating procedures used.
'''

#|hide
### Imports

In [30]:
#| export
import numpy as np

The following rows from an OPUS Report are decoded.
```
          1         2         3         4         5         6         7         8
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890
      USER: abcdefgh@XXXXXXXXX                       DATE: May 20, 2024
RINEX FILE: sfkw121p.24o                             TIME: 03:53:37 UTC
 EPHEMERIS: igs23122.eph [precise]                   STOP: 2024/04/30  22:51:00
  ANT NAME: TRMR10-2        NONE              # FIXED AMB:    65 /    68   :  96%
ARP HEIGHT: 0.000                             OVERALL RMS: 0.014(m)
 REF FRAME: NAD_83(2011)(EPOCH:2010.0000)              ITRF2014 (EPOCH:2024.3301)
         X:     -2475273.328(m)   0.019(m)          -2475274.384(m)   0.019(m)
         Y:     -4509243.734(m)   0.013(m)          -4509242.286(m)   0.013(m)
         Z:      3760194.885(m)   0.015(m)           3760194.828(m)   0.015(m)
       LAT:   37 56  3.46294      0.001(m)        37 56  3.49404      0.001(m)
     W LON:  118 45 49.83289      0.012(m)       118 45 49.89794      0.012(m)
    EL HGT:         1094.647(m)   0.012(m)              1094.000(m)   0.012(m)
                         UTM (Zone 11)         SPC (0404 CA 4)
Northing (Y) [meters]     4024219.772           612842.492
Easting (X)  [meters]      341727.091          2021197.302
US NATIONAL GRID DESIGNATOR: 11SLA4172724220(NAD 83)
          1         2         3         4         5         6         7         8
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890
```

#|hide
## Functions.

#|hide
### def decode_OPUS()

In [31]:
#| export
def decode_OPUS(
  txt='',        # A string containing the contents of an entire OPUS report.
  opus_file=''   # A filename of an OPUS Report.
) -> object:      # Decoded class variable of the OPUS report.
  '''
  
  '''
  if txt:
    opus_txt = txt
  elif opus_file:
    with open(opus_file, 'r') as f:
      opus_txt = f.read()
  else:
    raise Exception('No input provided.')
  class POINT:
    pass
  class OPUS:
    pass
  class STRINGS:
    pass
  opus = OPUS()
  opus.nad83 = POINT()
  opus.itrf  = POINT()
  opus.strings = STRINGS()
  for row in opus_txt.split('\n'):
    if row[23:30].strip() == 'UTM (':
      opus.strings.utm = row[34:37].strip()
      opus.utm_zone = int( opus.strings.utm )

    elif row[0:9].strip() == 'Northing':
      opus.strings.northing = row[23:37]
      opus.northing = float( opus.strings.northing )

    elif row[0:8].strip() == 'Easting':
      opus.strings.easting = row[23:37]
      opus.easting = float( opus.strings.easting )

    elif row[0:12].strip() == 'USER:':
      opus.user    = row[11:50].strip()
      opus.datestr = row[58:73].strip()

    elif row[0:12].strip() == 'RINEX FILE:':
      opus.rinex_file = row[11:25].strip()
      opus.timestr    = row[58:73].strip()

    elif row[0:12].strip() == 'EPHEMERIS:':
      opus.ephermis = row[25:38].strip('[] ')

    elif row[0:12].strip() == 'ANT NAME:':
      opus.ant_name = row[11:42].strip()

    elif row[0:12].strip() == 'ARP HEIGHT:':
      opus.strings.arp_height = row[11:17].strip()
      opus.arp_height = float( opus.strings.arp_height )

    elif row[0:12].strip() == 'REF FRAME:':
      opus.ref_frame = row[11:81].strip()

    elif row[0:12].strip() ==   'X:':
      opus.strings.nad83_x = row[15:28]
      opus.nad83.x = float( opus.strings.nad83_x )
      opus.strings.itrf_x = row[51:64]
      opus.itrf.x = float( opus.strings.itrf_x )

    elif row[0:12].strip() ==   'Y:':
      opus.strings.nad83_y = row[15:28]
      opus.nad83.y = float( opus.strings.nad83_y )
      opus.strings.itrf_y = row[51:64]
      opus.itrf.y = float( opus.strings.itrf_y )

    elif row[0:12].strip() ==   'Z:':
      opus.strings.nad83_z = row[15:28]
      opus.nad83.z = float( opus.strings.nad83_z )
      opus.strings.itrf_z = row[51:64]
      opus.itrf.z = float( opus.strings.itrf_z )

    elif row[0:12].strip() == 'LAT:':
      st = opus.strings.nad83_lat = row[12:16], row[16:19], row[20:28]
      opus.nad83.lat = np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])
      st = opus.strings.itrf_lat  = row[49:52], row[53:55], row[56:64]
      opus.itrf.lat =np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])

    elif row[0:12].strip() == 'W LON:':
      st = opus.strings.nad83_lon = row[12:16], row[16:19], row[20:28]
      opus.nad83.lon = -np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])
      st = opus.strings.itrf_lon = row[49:52], row[53:55], row[56:64]
      opus.itrf.lon = -np.sum( [float(st[0]), float(st[1])/60, float( st[2])/3600.0])

    elif row[0:12].strip() == 'EL HGT:':
      st = opus.strings.nad83_el_hgt = row[19:28]
      opus.nad83.el_hgt = float(st)
      st = opus.strings.itrf_el_hgt  = row[55:64]
      opus.itrf.el_hgt = float(st)

    elif row[0:12].strip() == 'ORTHO HGT:':
      st = opus.strings.navd88_hgt = row[19:28]
      opus.nad83.navd88_hgt = float(st)
      #st = row[55:64]

    elif row[0:28].strip() == 'US NATIONAL GRID DESIGNATOR:':
      opus.us_grid = row[28:44].strip()
  return opus

In [32]:
# Test with the entire OPUS file in a string.
opus = decode_opus(txt=opus_txt)
for k in opus.strings.__dict__:
  print(f'{k:24s} = {opus.strings.__dict__[k]}')
display(opus.__dict__, opus.nad83.__dict__, opus.itrf.__dict__)

arp_height               = 0.000
nad83_x                  =  -2475273.328
itrf_x                   =  -2475274.384
nad83_y                  =  -4509243.734
itrf_y                   =  -4509242.286
nad83_z                  =   3760194.885
itrf_z                   =   3760194.828
nad83_lat                = ('  36', ' 21', ' 0.18896')
itrf_lat                 = (' 36', '21', ' 0.20210')
nad83_lon                = (' 118', ' 45', '49.83289')
itrf_lon                 = ('118', '45', '49.89794')
nad83_el_hgt             =  1094.647
itrf_el_hgt              =  1094.000
navd88_hgt               =  1122.958
utm                      = 11
northing                 =    4024219.772
easting                  =     341727.091


{'nad83': <__main__.decode_opus.<locals>.POINT at 0x7fdb1066b390>,
 'itrf': <__main__.decode_opus.<locals>.POINT at 0x7fdb1066a590>,
 'strings': <__main__.decode_opus.<locals>.STRINGS at 0x7fdb3c318050>,
 'user': 'lidar532@gmail.com',
 'datestr': 'May 20, 2024',
 'rinex_file': 'sfkw121p.24o',
 'timestr': '03:53:37 UTC',
 'ephermis': 'precise',
 'ant_name': 'TRMR10-2        NONE',
 'arp_height': 0.0,
 'ref_frame': 'NAD_83(2011)(EPOCH:2010.0000)              ITRF2014 (EPOCH:2024.3301)',
 'utm_zone': 11,
 'northing': 4024219.772,
 'easting': 341727.091,
 'us_grid': '11SLA4172724220'}

{'x': -2475273.328,
 'y': -4509243.734,
 'z': 3760194.885,
 'lat': 36.35005248888889,
 'lon': -118.76384246944444,
 'el_hgt': 1094.647,
 'navd88_hgt': 1122.958}

{'x': -2475274.384,
 'y': -4509242.286,
 'z': 3760194.828,
 'lat': 36.35005613888889,
 'lon': -118.76386053888889,
 'el_hgt': 1094.0}

In [33]:
# Test with an OPUS file.
opus = decode_opus( opus_file = '../test_data/OPUS-california.txt' )

print("\n==========[ Extracted OPUS Data Strings =======")
for k in opus.strings.__dict__:
  print(f'{k:24s} = {opus.strings.__dict__[k]}')

print("\n==========[ Extracted Common OPUS Data  ]======")
for k in opus.__dict__:
  print(f'{k:24s} = {opus.__dict__[k]}')

print("\n==========[ Extracted NAD83 data ]=============")
for k in opus.nad83.__dict__:
  print(f'{k:24s} = {opus.nad83.__dict__[k]}')

print("\n==========[ Extracted ITRF data ]==============")
for k in opus.itrf.__dict__:
  print(f'{k:24s} = {opus.itrf.__dict__[k]}')
  


==========[ Extracted OPUS Data Strings =======
arp_height               = 0.000
nad83_x                  =  -2475273.328
itrf_x                   =  -2475274.384
nad83_y                  =  -4509243.734
itrf_y                   =  -4509242.286
nad83_z                  =   3760194.885
itrf_z                   =   3760194.828
nad83_lat                = ('  36', ' 21', ' 0.18896')
itrf_lat                 = (' 36', '21', ' 0.20210')
nad83_lon                = (' 118', ' 45', '49.83289')
itrf_lon                 = ('118', '45', '49.89794')
nad83_el_hgt             =  1094.647
itrf_el_hgt              =  1094.000
navd88_hgt               =  1122.958
utm                      = 11
northing                 =    4024219.772
easting                  =     341727.091

==========[ Extracted Common OPUS Data  ]======
nad83                    = <__main__.decode_opus.<locals>.POINT object at 0x7fdb106a0690>
itrf                     = <__main__.decode_opus.<locals>.POINT object at 0x7fdb106a1510>
st